# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google Cloud Storage

下記コードでGCPに接続

In [ ]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## 環境変数

In [ ]:
RESAS_API_KEY = '	02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

# 都道府県一覧

In [ ]:
import json
import urllib.request

def get_resas_preflist():
    url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    return res['result']

In [ ]:
preflist = get_resas_preflist()
print(preflist)

GoogleDriveとGoogleCloudStorageに保存

In [ ]:
# 保存先パスの設定
path = "resas/preflist.json"

# Google Driveへ保存
drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)

with open(drive_path, 'w') as f:
    json.dump(preflist,f,ensure_ascii=False)

# GoogleCloudStorage
gcs_path = "statistics-hyogo/{}".format(path)

with open(gcs_path, 'w') as f:
    json.dump(preflist,f,ensure_ascii=False)

# 市区町村一覧

In [ ]:
import json
import urllib.request

def get_resas_citylist(prefCode=0,designatedCity='all'):
  
    # 都道府県'prefCode'を指定
    # 0の場合は全都道府県の市区町村
    if prefCode == 0:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    else:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}'.format(prefCode)
    
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())


    # 政令指定都市　の指定　'join':政令指定都市統合　'split'：政令指定都市分割 'all'：全て
    if designatedCity == 'join':
      return list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))
    elif designatedCity == 'split':
      return list(filter(lambda x: x['bigCityFlag'] != '2', res['result']))
    else:
      return res['result']

In [ ]:
city1 = get_resas_citylist()
print(city1)

city2 = get_resas_citylist(designatedCity='join')
print(city2)

city3 = get_resas_citylist(prefCode=28,designatedCity='split')
print(city3)

city4 = get_resas_citylist(prefCode=28)
print(city4)

GoogleDriveとGoogleCloudStorageに保存

In [ ]:
citylist = get_resas_citylist()

# 保存先パスの設定
path = "resas/citylist.json"

# Google Driveへ保存
drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)

with open(drive_path, 'w') as f:
    json.dump(citylist,f,ensure_ascii=False)

# GoogleCloudStorage
gcs_path = "statistics-hyogo/{}".format(path)

with open(gcs_path, 'w') as f:
    json.dump(citylist,f,ensure_ascii=False)

# estat-APIの地域一覧

文字列5桁の地域コードに変換する

In [ ]:
import pandas as pd

def get_estat_areas():
  # 元データ
  df_preflist = pd.DataFrame(get_resas_preflist())
  df_citylist = pd.DataFrame(get_resas_citylist())
  
  # 市区町村の処理
  df_citylist = pd.merge(df_preflist, df_citylist, on='prefCode')
  df_citylist['areaCode'] = df_citylist.apply(lambda x: x['cityCode'], 1)
  df_citylist['areaName'] = df_citylist.apply(lambda x: x['cityName'].replace(x['prefName'], ''), 1)
  df_citylist['governmentType'] = 'city'
  
  # 都道府県の処理
  df_preflist['areaCode'] = df_preflist.apply(lambda x: str(x['prefCode']).zfill(5), 1)
  df_preflist['areaName'] = df_preflist.apply(lambda x: x['prefName'], 1)
  df_preflist['governmentType'] = 'prefecture'


  df = pd.concat([df_citylist, df_preflist], axis=0)

  return df

GoogleDriveとGoogleCloudStorageに保存

In [ ]:
def save_json(df,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)
  df.reset_index().to_json(drive_path,orient='records',force_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-hyogo/{}".format(path)
  df.reset_index().to_json(gcs_path,orient='records',force_ascii=False)

In [ ]:
df = get_estat_areas()
path = "resas/arealist.json"

save_json(df,path)